# Fourier Transform - from Density to Scattering

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from molass.Shapes import Sphere
from molass.DensitySpace import VoxelSpace
from molass.SAXS.Simulator import compute_saxs
from molass.SAXS.Models.Formfactors import homogeneous_sphere
%matplotlib widget

q = np.linspace(0.005, 0.7, 100)
R = 30

I = homogeneous_sphere(q, R)**2

sphere = Sphere(radius=10)
space = VoxelSpace(64, sphere)
saxs = compute_saxs(space.rho, q=q, dmax=64)

fig = plt.figure(figsize=(12, 5))
fig.suptitle('SAXS Simulation of a Homogeneous Sphere')
ax1 = fig.add_subplot(121, projection='3d')
ax1.set_title('Electron density')
space.plot_as_dots(ax1)

ax2 = fig.add_subplot(122)
ax2.set_title('SAXS intensity')
ax2.set_yscale('log')
ax2.plot(q, I, label='Analitical formula')
curve = saxs.get_curve()
ax2.plot(q, curve.y, label='Fourier transform simulation')
ax2.legend()
ax2.set_xlabel('q (1/Å)')

In [ ]:
from importlib import reload
import molass.SAXS.Models.Formfactors
reload(molass.SAXS.Models.Formfactors)
from molass.SAXS.Models.Formfactors import tri_axial_ellipsoid
I = tri_axial_ellipsoid(q, 30, 24, 18)**2
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 5))
ax2.set_yscale('log')
for ax in ax1, ax2:
    ax.plot(q, I, label='tri_axial_ellipsoid')

In [ ]:
from molass.Shapes import Ellipsoid
from molass.DensitySpace import VoxelSpace
from molass.SAXS.Simulator import compute_saxs, draw_saxs

ellipsoid = Ellipsoid(10, 8, 6)
space = VoxelSpace(64, ellipsoid)
saxs = compute_saxs(space.rho, q=q, dmax=64)

fig = plt.figure(figsize=(12, 5))
fig.suptitle('SAXS Simulation of a Tri-axial Ellipsoid')
ax1 = fig.add_subplot(121, projection='3d')
ax1.set_title('Electron density')
space.plot_as_dots(ax1)

ax2 = fig.add_subplot(122)
ax2.set_title('SAXS intensity')
ax2.set_yscale('log')
ax2.plot(q, I, label='Tri-axial Ellipsoid')
curve = saxs.get_curve()
ax2.plot(q, curve.y, label='Fourier transform simulation')
ax2.legend()
ax2.set_xlabel('q (1/Å)')